### CS5228 Mini Project Team 07 - Data Preprocessing

#### Objective

This notebook aims to preprocess the raw dataset to:
- Handle missing values and perform data cleaning
- Encode categorical features
- Normalize numerical features

The output of this notebook create a pickle file `cleaned_datasets.pkl` containing variants of train and test datasets. Structure:
```python
{
    "standard_scale": 
    {
        "train": train_df_with_standard_scaling,
        "test": test_df_with_standard_scaling
    },

    "min_max_scale": 
    {
        "train": train_df_with_min_max_scaling,
        "test": test_df_with_min_max_scaling
    } 
}
```

#### Imports

In [100]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Read the raw dataset

In [101]:
train_df = pd.read_csv('./dataset/churn-bigml-80.csv')
test_df = pd.read_csv('./dataset/churn-bigml-20.csv')

In [102]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [103]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   667 non-null    object 
 1   Account length          667 non-null    int64  
 2   Area code               667 non-null    int64  
 3   International plan      667 non-null    object 
 4   Voice mail plan         667 non-null    object 
 5   Number vmail messages   667 non-null    int64  
 6   Total day minutes       667 non-null    float64
 7   Total day calls         667 non-null    int64  
 8   Total day charge        667 non-null    float64
 9   Total eve minutes       667 non-null    float64
 10  Total eve calls         667 non-null    int64  
 11  Total eve charge        667 non-null    float64
 12  Total night minutes     667 non-null    float64
 13  Total night calls       667 non-null    int64  
 14  Total night charge      667 non-null    fl

Based on the overview above, there is no missing entry in both train and test datasets. Now we split numerical and categorical features for further EDA and data preprocessing.

Numerical columns: 'Account length ', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls'

Categorical columns: 'State', 'Area code', 'International plan', 'Voice mail plan', 'Churn'(which is the target column)

In [104]:
train_df.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [105]:
numerical_columns = ['Account length', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls']
categorical_columns = ['State', 'Area code', 'International plan', 'Voice mail plan', 'Churn']

In [106]:
train_df.head(5)

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


#### Categorical feature preprocessing

Observation: 

For the Area code feature, we can perform OHE given it only has 3 unique values and they appear in both train and test sets.

Since the State feature contains 51 unique values, applying One-Hot Encoding (OHE) would result in a large number of new features. Given that our dataset has only 2,666 entries, this could lead to the curse of dimensionality and potential overfitting. To mitigate this, two alternative encoding methods are considered:

- Label Encoding: Assigns a unique numeric value to each state. However, this method may introduce an unintended ordinal relationship among categorical values.
- Frequency Encoding: Replaces each state with its relative occurrence in the dataset, capturing its distribution without implying any order.

To avoid data leakage in test dataset:
- Label Encoding: The mapping derived from the training set should be applied to the test set, and any unseen categories should be assigned a default value 0.
- Frequency Encoding: The frequency mapping from the training set should be used for the test set, and any unseen states should be assigned the mean frequency from the training distribution.


However, these encoding methods also have their flaws: label encoding may [unnecessarily increase the model complexity and increase the chances of overfitting](https://stackoverflow.com/questions/61217713/labelencoder-for-categorical-features), and frequency encoding diminishes the distinctiveness of a feature to a large extent. As such we can introduce a feature called `state_churn_mode` and `state_churn_prob` to indicate the mode and probabilty of churning in the traing set for each state.

In [107]:
train_df['Area code'].unique(), train_df['State'].unique().size

(array([415, 408, 510], dtype=int64), 51)

In [108]:
test_df['Area code'].unique(), test_df['State'].unique().size

(array([408, 415, 510], dtype=int64), 51)

Functions to encode categorical features

In [109]:
def perform_binary_encoding(train, test, columns):
    for column in columns:
        train[column] = train[column].map({'Yes': 1, 'No': 0})
        test[column] = test[column].map({'Yes': 1, 'No': 0})
    return train, test

def perform_one_hot_encoding(train, test, columns):
    train = pd.get_dummies(train, columns=columns)
    test = pd.get_dummies(test, columns=columns)
    return train, test

def perform_frequency_encoding(train, test, columns):
    for column in columns:
        new_col_name = column + '_freq'
        freq_encoding = train[column].value_counts(normalize=True)
        train[new_col_name] = train[column].map(freq_encoding)
        test[new_col_name] = test[column].map(freq_encoding)
        test[new_col_name].fillna(freq_encoding.mean(), inplace=True)
    return train, test

def perform_label_encoding(train, test, columns):
    for column in columns:
        new_col_name = column + '_label'
        label_encoding = {k: i for i, k in enumerate(train[column].unique(), 0)}
        train[new_col_name] = train[column].map(label_encoding)
        test[new_col_name] = test[column].map(label_encoding)
    return train, test

def perform_statistics_encoding(train, test, columns, target):
    target_lower = target.lower()  # Convert target to lowercase for consistency
    
    for column in columns:
        churn_prob = train.groupby(column)[target].mean()

        prob_col = f"{column}_{target_lower}_prob"

        # Map the computed values from train to both train and test, to avoid data leakage
        train[prob_col] = train[column].map(churn_prob)  
        test[prob_col] = test[column].map(churn_prob)

    return train, test

def drop_columns(train, test, columns):
    return train.drop(columns=columns, axis=1), test.drop(columns=columns, axis=1)

def rearrange_columns(train, test, columns, target, tags):
    for column in columns:
        cols = train.columns.tolist()
        raw_idx = cols.index(column)

        # Iterate over tags and adjust the column positions
        for i, tag in enumerate(tags, start=1):
            tagged_col = f"{tag}{column}" if tag.endswith('_') else f"{column}{tag}"
            if tagged_col in cols:
                cols.insert(raw_idx + i, cols.pop(cols.index(tagged_col)))

        train = train[cols]
        test = test[cols]

    # Move target column to the end
    cols = train.columns.tolist()
    cols.insert(len(cols), cols.pop(cols.index(target)))
    train = train[cols]
    test = test[cols]

    return train, test

In [110]:
train_df, test_df = perform_binary_encoding(train_df, test_df, ['International plan', 'Voice mail plan'])
train_df, test_df = perform_one_hot_encoding(train_df, test_df, ['Area code'])
train_df, test_df = perform_frequency_encoding(train_df, test_df, ['State'])
# train_df, test_df = perform_label_encoding(train_df, test_df, ['State'])
train_df, test_df = perform_statistics_encoding(train_df, test_df, ['State'], target="Churn")
train_df, test_df = rearrange_columns(train_df, test_df, ['State'], 'Churn', ["_freq", "_churn_prob"])
train_df, test_df = drop_columns(train_df, test_df, ['State'])


#### Numerical feature preprocessing

To avoid data leakage, we fit the scaler only on the training set and use the fit scaler to transform both train and test sets.

Function to normalize numerical features

In [111]:
def perform_scaling(train, test, columns, scaler):
    train[columns] = scaler.fit_transform(train[columns])
    test[columns] = scaler.transform(test[columns])
    return train, test

In [112]:
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

train_df_standard, test_df_standard = perform_scaling(train_df.copy(), test_df.copy(), numerical_columns, standard_scaler)
train_df_min_max, test_df_min_max = perform_scaling(train_df.copy(), test_df.copy(), numerical_columns, min_max_scaler)

In [113]:
train_df_min_max.head()

,State_freq,State_churn_prob,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,...,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Area code_408,Area code_415,Area code_510,Churn
0,0.019505,0.192308,0.524793,0,1,0.50,0.755701,0.68750,0.755701,0.542755,...,0.436090,0.572152,0.500,0.15,0.500000,0.111111,False,True,False,False
1,0.024756,0.151515,0.438017,0,1,0.52,0.460661,0.76875,0.460597,0.537531,...,0.526316,0.600000,0.685,0.15,0.685185,0.111111,False,True,False,False
2,0.018755,0.280000,0.561983,0,0,0.00,0.693843,0.71250,0.693830,0.333242,...,0.533835,0.338608,0.610,0.25,0.609259,0.000000,False,True,False,False
3,0.024756,0.151515,0.342975,1,0,0.00,0.853478,0.44375,0.853454,0.170195,...,0.421053,0.436076,0.330,0.35,0.329630,0.222222,True,False,False,False
4,0.019505,0.134615,0.305785,1,0,0.00,0.475200,0.70625,0.475184,0.407754,...,0.661654,0.407595,0.505,0.15,0.505556,0.333333,False,True,False,False


In [114]:
train_df_standard.head()

,State_freq,State_churn_prob,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,...,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Area code_408,Area code_415,Area code_510,Churn
0,0.019505,0.192308,0.692163,0,1,1.247508,1.579670,0.484868,1.579942,-0.058619,...,-0.469031,0.856707,-0.085020,-0.597527,-0.085681,-0.429172,False,True,False,False
1,0.024756,0.151515,0.161278,0,1,1.320985,-0.329918,1.135375,-0.330194,-0.095916,...,0.149054,1.049293,1.242179,-0.597527,1.242921,-0.429172,False,True,False,False
2,0.018755,0.280000,0.919686,0,0,-0.589414,1.179302,0.685024,1.179465,-1.554439,...,0.200561,-0.758391,0.704125,0.216894,0.698194,-1.191955,False,True,False,False
3,0.024756,0.151515,-0.420168,1,0,-0.589414,2.212509,-1.466653,2.212675,-2.718509,...,-0.572045,-0.084339,-1.304609,1.031314,-1.307995,0.333610,True,False,False,False
4,0.019505,0.134615,-0.647691,1,0,-0.589414,-0.235822,0.634985,-0.235772,-1.022461,...,1.076181,-0.281303,-0.049150,-0.597527,-0.045823,1.096392,False,True,False,False


Save the preprocessed df as a pickle file

In [115]:
data_dict = {
    "standard_scale": 
    {
        "train": train_df_standard,
        "test": test_df_standard
    },

    "min_max_scale": 
    {
        "train": train_df_min_max,
        "test": test_df_min_max
    } 
}

with open('./dataset/cleaned_datasets.pkl', 'wb') as f:
    pickle.dump(data_dict, f)